## Prompts Exploratory Data Analysis
Using this notebook, we can analyze and extract key insights from the prompts used for training our LLM model.

#### EDA Overview
Token Wise:
- Average Tokens Length
- Token Lengths Distribution
- Min/Max in tokens

Coverage:
- Prompt Coverage as a function of Tokens Lenght
- Prompt Coverage as a function of hardcoded LLM context size.

Word Analysis:
- Word Cloud
---

Insights:
- `Average Tokens Length` : see wether our prompts are balanced in relation to model context size.
- `Token Lengths Distribution` : see variety of sentence lengths, skewed towards short or long sentences
- `Min/Max in tokens` : anomalies or outliers in the training set

- `Prompt Coverage w.r.t Tokens Lenght` : how effective the prompts cover the entire range of tokens lengths
- `Word Cloud` : main topics and subjects within the dataset

In [ ]:
# Dataset Prerequisites
# 1. Load dataset, process each prompt such that it is a raw string.
# 2. Load tokenizer, set SOL, EOS
# 3. Add SOL/EOS to each prompt

### Prompt [1] Average Tokens Length


In [ ]:
# Tokenize each prompt
# Gather token lengths

### Prompt [2] Token Lengths Distribution

In [ ]:
# Plot tokens distributions

### Prompt [3] Min/Max in Tokens

In [ ]:
# Plot percentiles .25 .5 .75

### Coverage [1] Prompt Coverage as a function of Tokens Length

In [ ]:
# Plot gaussian bell

### Coverage [2] Prompt Coverage as a function of hardcoded context size

In [ ]:
# Plot prompts coverage in relation to tokens count

### Word Analysis [1] World Cloud

In [ ]:
# Plot word cloud distribution